# Download PointData for Magnaprobe data within bounding box 

### Get database session 

In [3]:
# Import the function to get connect to the db
from snowexsql.db import get_db

# Import our class for the points table
from snowexsql.data import PointData #only pull in point data 

# Import a useful function to format that data into a dataframe
from snowexsql.conversions import query_to_geopandas

# Import some tools to build dates 
from datetime import date
import pandas as pd
from sqlalchemy import inspect

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

In [48]:
# database imports
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas, query_to_pandas
from snowexsql.db import get_table_attributes

from sqlalchemy.sql import func

import shapely.geometry
from geoalchemy2.shape import from_shape
import geoalchemy2.functions as gfunc

# load the database
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
engine, session = get_db(db_name)

print('SnowEx Database successfully loaded!')

SnowEx Database successfully loaded!


### Query the DB to see what tables are available

In [4]:
# Using the function get_db, we receive 2 ways to interact with the database
engine, session = get_db(db_name)

# Output the list of tables in the database 
tables = inspect(engine).get_table_names()
print(tables)
# spatial_ref_sys doesn't need to be used, just contains the spatial reference data for the points (under the hood) 

session.close()

['spatial_ref_sys', 'points', 'layers', 'sites', 'images']


Import classes that reflect this data 

In [5]:
from snowexsql.data import PointData, LayerData, ImageData, SiteData
# PointsData is any kind of data that can be shown as a single point (e.g., GPR) 
# LayerData any data the can be represented by a single point and a single value over a range of depth 
# ImageData is raster data, rasters all broken into tiles 
# SiteData is metadata for pits, gets repeated for every record so an individual table was made for it 

### Query a table to see what columns to use 

In [63]:
# Import the function to investigate a table
# from snowexsql.db import get_table_attributes 

# Use the function to see what columns are available to use. 
# db_columns = get_table_attributes(SiteData) ###################### change this one to the desired data type 

# Print out the results nicely
# print("These are the available columns in the table:\n \n* {}\n".format('\n* '.join(db_columns)))


### Add Bounding Box 

In [54]:
# How to pull out ALL point data from the database that falls within our box
bbox_WSEN = 742000, 4322000, 747000, 4325000 # EPSG 26912?
x1, y1, x2, y2 = bbox_WSEN
polygon = shapely.geometry.Polygon([[x1, y1], [x1, y2], [x2, y2], [x2, y1]]) # used box() before
wkb_element = from_shape(polygon, srid=26912) # which srid is right?


### Filter Data

In [61]:
# Site name
site_name = "Grand Mesa"

# Instrument name 
instrument = "magnaprobe" 

# Get a session
engine, session = get_db(db_name)

# The part inside the query function is what we want back, in this case all columns for the point data
qry = session.query(PointData.geom, PointData.easting, PointData.northing, PointData.date, PointData.value, 
                    PointData.type, PointData.instrument)

# Filter by bounding box 
qry = qry.filter(gfunc.ST_Within(PointData.geom, wkb_element))

# Filter by site
# qry = qry.filter(PointData.site_name == site_name)

# Filter by an instrument 
qry = qry.filter(PointData.instrument == instrument) # .in_(['magnaprobe', 'pit_ruler']))

# Execute the query and convert to geopandas in one handy function
df = query_to_geopandas(qry, engine) #directly pass to geopandas dataframe
print(df.head())

# how many did we retrieve?
print(f'{len(df.index)} records returned!')

session.close()

                             geom        easting      northing        date  \
0  POINT (742381.442 4324179.239)  742381.442244  4.324179e+06  2020-01-28   
1  POINT (742377.774 4324185.793)  742377.774179  4.324186e+06  2020-01-28   
2  POINT (742378.640 4324185.819)  742378.639908  4.324186e+06  2020-01-28   
3  POINT (742376.043 4324185.739)  742376.042720  4.324186e+06  2020-01-28   
4  POINT (742376.908 4324185.766)  742376.908450  4.324186e+06  2020-01-28   

   value   type  instrument  
0   97.0  depth  magnaprobe  
1  123.0  depth  magnaprobe  
2  127.0  depth  magnaprobe  
3  117.0  depth  magnaprobe  
4  122.0  depth  magnaprobe  
24029 records returned!


# IGNORE EVERYTHING PAST THIS POINT

### Create a Query

In [7]:
# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

# Lets grab a single row from the points table
qry = session.query(PointData)
qry = qry.filter(PointData.site_name == 'Grand Mesa')
count = qry.count() 
print(count) 

# Execute that query!
# result = qry.all()

session.close()

6940828


In [46]:
# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name) # try to bracket this with session.close() as closely as possible as not to be kicked from the database 

# Its convenient to store a query like the following 
qry = session.query(PointData.instrument, PointData.date, PointData.site_name) 

# Then filter on it to just density profiles
qry = qry.filter(PointData.site_name == 'Grand Mesa') # pull data from location 
qry = qry.filter(PointData.instrument == 'magnaprobe') 
# qry = qry.filter(PointData.instrument == 'pit ruler') 
# qry = qry.filter(PointData.instrument == 'Mala 1600 MHz GPR') 
# qry = qry.filter(PointData.instrument == 'Mala 1600 MHz GPR') 
# qry = qry.filter(PointData.instrument == 'pulse EKKO Pro multi-polarization 1 GHz GPR') 


# protect ourselves from a lot of data
# qry = qry.limit(500) # only pull 5 points 

result = qry.all()
# print(result)

check = qry.filter(PointData.instrument == 'pit ruler')
print(check)

# Let see what instruments are available 
result = session.query(PointData.instrument).filter(PointData.type == 'depth').distinct().all()
print('\nInstruments Used to Measure Depth: ', result)

result = session.query(PointData.instrument).filter(PointData.type == 'two_way_travel').distinct().all()
print('\nInstruments Used to Measure Two Way Travel: ', result)

result = session.query(PointData.instrument).filter(PointData.type == 'swe').distinct().all()
print('\nInstruments Used to Measure SWE: ', result)

result = session.query(PointData.type).distinct().all()
print('\nInstrument Measurement Types: ', result)

result = session.query(PointData.instrument).distinct().all()
print('\nInstrument Types: ', result)

session.close()

SELECT public.points.instrument AS public_points_instrument, public.points.date AS public_points_date, public.points.site_name AS public_points_site_name 
FROM public.points 
WHERE public.points.site_name = %(site_name_1)s AND public.points.instrument = %(instrument_1)s AND public.points.instrument = %(instrument_2)s

Instruments Used to Measure Depth:  [(None,), ('Mala 1600 MHz GPR',), ('Mala 800 MHz GPR',), ('pulse EKKO Pro multi-polarization 1 GHz GPR',), ('pit ruler',), ('mesa',), ('magnaprobe',), ('camera',)]

Instruments Used to Measure Two Way Travel:  [('Mala 1600 MHz GPR',), ('Mala 800 MHz GPR',), ('pulse EKKO Pro multi-polarization 1 GHz GPR',)]

Instruments Used to Measure SWE:  [('Mala 1600 MHz GPR',), ('Mala 800 MHz GPR',), ('pulse EKKO Pro multi-polarization 1 GHz GPR',)]

Instrument Measurement Types:  [('swe',), ('depth',), ('two_way_travel',)]

Instrument Types:  [(None,), ('Mala 1600 MHz GPR',), ('Mala 800 MHz GPR',), ('pulse EKKO Pro multi-polarization 1 GHz GPR',), 

### List Available Dates for Each Point Data Type 

In [9]:
# df = pd.DataFrame(columns = len(session.query(PointData.instrument).filter(PointData.type == 'depth').distinct().all()))

results = session.query(PointData.date).filter(PointData.instrument == 'magnaprobe').distinct().all()
print('\nAvailable Dates for Magnaprobe = {}'.format(', '.join([str(r[0]) for r in results])))

results = session.query(PointData.date).filter(PointData.instrument == 'Mala 1600 MHz GPR').distinct().all()
print('\nAvailable Dates Mala 1600 MHz GPR = {}'.format(', '.join([str(r[0]) for r in results])))

results = session.query(PointData.date).filter(PointData.instrument == 'Mala 800 MHz GPR').distinct().all()
print('\nAvailable Dates Mala 800 MHz GPR = {}'.format(', '.join([str(r[0]) for r in results])))

results = session.query(PointData.date).filter(PointData.instrument == 'pulse EKKO Pro multi-polarization 1 GHz GPR').distinct().all()
print('\nAvailable Dates pulse EKKO Pro multi-polarization 1 GHz GPR = {}'.format(', '.join([str(r[0]) for r in results])))

results = session.query(PointData.date).filter(PointData.instrument == 'pit ruler').distinct().all()
print('\nAvailable Dates pit ruler = {}'.format(', '.join([str(r[0]) for r in results])))



Available Dates for Magnaprobe = 2020-01-28, 2020-01-29, 2020-01-30, 2020-01-31, 2020-02-01, 2020-02-04, 2020-02-05, 2020-02-06, 2020-02-07, 2020-02-08, 2020-02-10, 2020-02-11, 2020-02-12

Available Dates Mala 1600 MHz GPR = 2020-02-04, 2020-02-06, 2020-02-05

Available Dates Mala 800 MHz GPR = 2020-01-30, 2020-02-01, 2020-01-31, 2020-01-28, 2020-01-29

Available Dates pulse EKKO Pro multi-polarization 1 GHz GPR = 2020-02-04, 2020-02-09, 2020-02-06, 2020-02-07, 2020-02-08, 2020-01-28, 2020-01-29

Available Dates pit ruler = 2019-12-19, 2020-01-22, 2020-01-28, 2020-01-29, 2020-01-30, 2020-01-31, 2020-02-01, 2020-02-03, 2020-02-04, 2020-02-05, 2020-02-06, 2020-02-08, 2020-02-09, 2020-02-10, 2020-02-11, 2020-02-12, 2020-03-12, 2020-03-18, 2020-03-21


In [50]:
results = session.query(SiteData.date).filter(SiteData.instrument == 'magnaprobe').distinct().all()
print('\nAvailable Dates for Magnaprobe = {}'.format(', '.join([str(r[0]) for r in results])))


AttributeError: type object 'SiteData' has no attribute 'instrument'